# Recommendation System

## Задание 1

In [1]:
import pandas as pd
import numpy as np

Объедини общие данные о фильмах [tmdb_5000_movies](https://files.sberdisk.ru/s/te4QbzdxKgsFQXA) и каст фильмов 
[tmdb_5000_credits](https://files.sberdisk.ru/s/H9oRuXQt5mFz3T9). Оставь в датасете только фильмы, которые вышли в "релиз".\
Выведи количество фильмов, оставшихся после фильтрации.

In [2]:
# Создаем датафремы из csv файлов
df_movies = pd.read_csv('../datasets/tmdb_5000_movies.csv')
df_credits = pd.read_csv('../datasets/tmdb_5000_credits.csv')

# Объединяем датафреймы в один, причем movies слева и объединение по столбцу id, credits справа и объединение по столбцу movie_id
df = pd.merge(df_movies, df_credits, how='left', left_on='id', right_on='movie_id')


In [3]:
# посмотрим какие столбцы получились в объединенном датафрейме
list(df)


['budget',
 'genres',
 'homepage',
 'id',
 'keywords',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title_x',
 'vote_average',
 'vote_count',
 'movie_id',
 'title_y',
 'cast',
 'crew']

In [4]:
# Оставим в датафреймы только те фильмы, которые вышли в релиз
df = df[df['status'] == 'Released']

# Посчитаем количество строк в датафрейме
print('Количество фильмов, оставшихся после фильтрации (которые уже вышли в релиз) =', df.shape[0])


Количество фильмов, оставшихся после фильтрации (которые уже вышли в релиз) = 4795


## Задание 2

Самый наивный подход к рекомендации фильмов - рекомендовать фильмы с лучшими оценками пользователей. Фильмы, которые пользуются большей популярностью и признанием критиков, с большей вероятностью понравятся среднему зрителю.

Для справедливой оценки фильмов возьмем текущую рейтинговую систему IMDB (weighted rating (WR)), которая рассчитывается по формуле:
$$WR = \frac{v}{v + m} ⋅ R + \frac{m}{v + m} ⋅ C$$ 
$v$ - количество голосов \
$m$ - количество голосов для включения в финальную таблицу \
$R$ - средняя оценка \
$C$ - средняя оценка всех фильмов 

Имплементируй функцию `weighted_rating`. С её помощью расcчитай рейтинг для каждого фильма и сохрани его в колонку `simple_score`.\
Выведи топ-5 фильмов по получившемуся рейтингу.
> В качестве параметра $m$ выбери 95-й квантиль количества голосов.

In [5]:
def weighted_rating():
    """Функция считает рейтинг по системе IMDB для конкретного фильма"""
    v = df['vote_count']
    m = np.percentile(df['vote_count'], 95) # 95% голосов попадет в этот диапазон
    R = df['vote_average']
    C = df['vote_average'].mean()
    return v *R / (v+m) + m *C / (v+m)


In [6]:
## Сохраняем рейтинг по системе IMDB для каждого фильма в новую колонку simple_score нашего датафрейма
df['simple_score'] = weighted_rating()


In [7]:
#продемонстрируем, что колонка simple_score создалась
list(df)


['budget',
 'genres',
 'homepage',
 'id',
 'keywords',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title_x',
 'vote_average',
 'vote_count',
 'movie_id',
 'title_y',
 'cast',
 'crew',
 'simple_score']

In [8]:
# Выводим топ 5 фильмов по получившемуся рейтингу
top5 = df.sort_values('simple_score', ascending=False)
top5['title_x'].head(5)


1881    The Shawshank Redemption
65               The Dark Knight
662                   Fight Club
96                     Inception
3232                Pulp Fiction
Name: title_x, dtype: object

## Задание 3

Такой подход к рекомендациям очень наивен, так как не учитывает информацию о самом фильме (жанр, режиссер, описание, актеры и т.п). \
**Content Based Filtering** (Фильтрация на основе содержания) - тип рекомендательной системы, которая предлагает пользователям похожие элементы на основе конкретного элемента. Общая идея этих рекомендательных систем заключается в том, что если человеку понравился определенный товар, то ему понравится и похожий на него товар.

<center><img src="../misc/images/content.png" alt= “” width="300" height="500">

Реализуем алгоритм рекомендации на основе описания фильма. Для это требуется провести предобработку текста:
* Замени NaN в описании фильма на пустой символ `''`
* Удали все английские стоп слова (используй параметр `stop_words` в `TfidfVectorizer`)
* Расcчитай [Tf-Idf](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) для описания фильма

Выведи размер получившейся матрицы Tf-Idf

> Для [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) используйте параметры по умолчанию 

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
vector = TfidfVectorizer(stop_words='english')  # создаем матрицу функций, удаляя все английские слова
df['overview'] = df['overview'].fillna('')  # заменяем NaN в описании фильма на пустой символ ""
matrix = vector.fit_transform(df.overview)  # создаем матрицу TF-Idf
print ('Размер получившейся матрицы Tf-Idf', matrix.shape)

Размер получившейся матрицы Tf-Idf (4795, 20970)


## Задание 4

Теперь тебе необходимо вычислить показатель сходства между описаниями фильмов. Используем косинусное расстояние, оно рассчитывается по формуле:
$$cos(Θ) = \frac{A ⋅ B}{∥A∥ ∥B∥} = \frac{ Σ_{i=1}^{n} A_i ⋅ B_i } { \sqrt{Σ_{i=1}^{n}A_{i}^{2}} ⋅ {\sqrt{Σ_{i=1}^{n}B_{i}^{2}}}}$$
Но поскольку мы использовали векторизатор TF-IDF на предыдущем шаге, достаточно вычислить скалярное произведение, которое и даст оценку косинусного сходства. Рассчитать его можно через [linear_kernel](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.linear_kernel.html). Результат сохрани в переменную `cosine_sim`.

Выведи размер получившейся матрицы `cosine_sim`.

In [11]:
from sklearn.metrics.pairwise import linear_kernel

In [12]:
cosine_sim = linear_kernel(matrix) # создаем квадратную матрицу схожести фильмов, у нее главная диагональ единица, потому что фильм сравнивается сам с собой

print('Размер получившейся матрицы cosine_sim', cosine_sim.shape)


Размер получившейся матрицы cosine_sim (4795, 4795)


## Задание 5

Напиши функцию `get_recommendations`. На вход она принимает:
* `movies_dataset` - датасет фильмов
* `title` - название фильма, для которого мы будем искать похожие
* `cosine_sim` - матрица расстояний между описаниями
* `top_k` - топ-k cхожих фильмов

Возвращает top_k названий фильмов, описание которых похоже на выбранный фильм.\
Выведи топ-5 фильмов для `title='Saving Private Ryan'`

In [19]:
def get_recommendations(movies_dataset, title, cosine_sim, top_k):
    """Функция поиска фильмов, максимально схожих с выбранным"""

    # создаем Series с названиями фильмов и индексами
    movie_index = pd.Series(movies_dataset.index, index = movies_dataset['title_x']).drop_duplicates()

    # получаем индекс фильма по его названию
    index = movie_index[title]

    # получаем список похожих фильмов по индексу
    similar = list(enumerate(cosine_sim[index]))

    # получаем отсортированный по убыванию список максимально похожих фильмов исключая самого себя (первый ильм со значением в матрице 1)
    sort = sorted(similar, key = lambda x: x[1], reverse=True)[1:]

    # получаем индексы похожих фильмов из top_k
    top_index = [score[0] for score in sort[0: top_k]]

    # получаем названи фильмов из топа по их индексам
    top_k_movies = movies_dataset['title_x'].iloc[top_index].values.tolist()

    return top_k_movies




In [21]:
get_recommendations(movies_dataset=df, title='Saving Private Ryan', cosine_sim=cosine_sim, top_k=5)


['The Great Raid',
 'The Monuments Men',
 'The Expendables 2',
 'Abandoned',
 'The Train']

## Задание 6

Еще один подход к построению рекомендательной системы - подход на основе сходства между пользователями. Этот подход называется **Collaborative Filtering** (Коллаборативная фильтрация).
<center><img src="../misc/images/all.png" alt= “” width="600" height="700"></center>
Коллаборативная фильтрация - это тип рекомендательной системы, которая использует поведение и предпочтения похожих пользователей для рекомендации товаров или продуктов конкретному пользователю. Система собирает данные о прошлом поведении пользователей, такие как покупки, рейтинги и отзывы, и анализирует их для выявления закономерностей и сходства между пользователями. На основе этих закономерностей система рекомендует товары, которые понравились или были приобретены другими такими же пользователями в прошлом.

Для реализации Коллаборативной фильтрации нам потребуются оценки пользователей [ratings](../datasets/ratings.csv).

>userId - id пользователя \
movieId - id фильма \
rating - оценка фильма (от 0 до 5)\
timestamp - время оценки


Воспользуйся библиотекой [surprise](https://surpriselib.com/) для обучения модели оценки рейтинга фильма [SVD](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD). Выведи средние значения 'RMSE', 'MAE' на кросс-валидации с параметрами `cv=5`.

In [30]:
# установка библиотеки surprise
#!pip install surprise
#!conda install -c conda-forge scikit-surprise


"conda" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [31]:
from surprise import Reader, Dataset, SVD

ModuleNotFoundError: No module named 'surprise'